<a href="https://colab.research.google.com/github/TooBrokeForEvo/NLPCV_Assessment/blob/main/Assignment_3091801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Environment and Data Setup


In [ ]:
#Building the Baseline Model


In [ ]:
#Experimentation and Enhancement
